In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('nba_games.csv', index_col=0)  # The index_col = 0 indicates that the first col of the csv file is the index.  
df.describe()

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,+/-,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,mp_max,mp_max.1,fg_max,fga_max,fg%_max,...,ortg_opp,drtg_opp,mp_max_opp,mp_max_opp.1,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,total_opp,home_opp,season
count,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,0.0,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.0,17772.000000,17772.000000,0.0,0.0,17772.000000,17772.000000,17772.000000,...,17772.000000,17772.000000,0.0,0.0,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000,17772.000000
mean,241.564258,241.564258,39.954817,87.020538,0.460049,11.022901,30.800923,0.356935,17.415710,22.627279,0.770461,10.098863,34.031904,44.130767,23.655413,7.645285,4.837497,13.447389,20.165485,108.348244,NaN,0.559665,0.523530,0.354005,0.263397,22.703714,77.297361,50.000867,59.126868,7.787936,8.644880,12.169508,100.0,110.617359,110.617359,NaN,NaN,9.767950,19.216802,0.820324,...,110.617359,110.617359,NaN,NaN,9.767950,19.216802,0.820324,3.804918,8.257090,0.778250,6.423250,7.849595,0.991401,3.726086,8.758609,11.273295,7.665204,2.577031,2.135269,4.011535,4.352071,26.599989,13.087553,0.938462,0.936513,0.827522,1.107340,19.926553,38.582759,25.095403,42.718473,5.937964,9.013302,40.079811,34.864979,178.187430,118.660758,108.348244,0.500000,2018.952284
std,7.002952,7.002952,5.225686,7.233960,0.054656,3.961903,7.905325,0.089744,5.969888,7.274001,0.102940,3.722156,5.410393,6.548409,5.175290,2.920185,2.479114,3.856099,4.253571,12.786684,NaN,0.061613,0.065951,0.086423,0.092654,7.208534,7.208577,5.257238,9.921340,2.902772,4.321392,3.374251,0.0,11.645548,11.645548,NaN,NaN,2.480479,4.244654,0.159069,...,11.645548,11.645548,NaN,NaN,2.480479,4.244654,0.159069,1.508266,2.579034,0.216759,2.801986,3.237968,0.043816,1.652267,2.544107,3.239086,2.678546,1.050831,1.146880,1.436492,0.909194,6.892052,8.950435,0.206218,0.218732,0.175385,0.812938,12.036766,13.735093,9.478922,14.801285,4.493377,6.725743,21.216193,8.505948,40.809205,11.087727,12.786684,0.500014,2.015800
min,240.000000,240.000000,19.000000,60.000000,0.269000,0.000000,4.000000,0.000000,1.000000,1.000000,0.143000,0.000000,16.000000,22.000000,7.000000,0.000000,0.000000,1.000000,6.000000,64.000000,NaN,0.329000,0.286000,0.045000,0.012000,0.000000,47.500000,31.500000,21.200000,0.000000,0.000000,1.000000,100.0,68.200000,68.200000,NaN,NaN,3.000000,8.000000,0.350000,...,68.200000,68.200000,NaN,NaN,3.000000,8.000000,0.350000,0.000000,1.000000,0.000000,1.000000,1.000000,0.400000,0.000000,3.000000,4.000000,2.000000,0.000000,0.000000,1.000000,2.000000,10.000000,-18.000000,0.500000,0.417000,0.200000,0.067000,0.000000,16.500000,9.100000,12.800000,0.000000,0.000000,4.600000,22.100000,89.000000,77.000000,64.000000,0.000000,2016.000000
25%,240.000000,240.000000,36.000000,82.000000,0.422000,8.000000,25.000000,0.297000,13.000000,17.000000,0.706000,7.000000,30.000000,40.000000,20.00

Since this is time-series data we will sort the dataframe by the date, similar to what was done in the tutorial. 

In [6]:
df = df.sort_values("date")
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,+/-,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,mp_max,mp_max.1,fg_max,fga_max,fg%_max,...,mp_max_opp.1,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
16086,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,25.0,33.0,21.0,9.0,3.0,18.0,26.0,95.0,NaN,0.501,0.458,0.217,0.325,18.6,54.3,37.1,60.0,9.0,4.5,15.9,100.0,94.9,110.9,NaN,NaN,7.0,20.0,1.000,...,NaN,14.0,26.0,1.000,5.0,12.0,1.000,7.0,7.0,1.0,5.0,5.0,9.0,7.0,2.0,2.0,7.0,5.0,40.0,20.0,1.136,1.000,0.462,0.571,39.0,27.2,25.2,50.1,19.1,7.9,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
16904,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,NaN,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,NaN,NaN,12.0,22.0,0.571,...,NaN,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
16905,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,NaN,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,NaN,NaN,8.0,22.0,0.750,...,NaN,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
16087,240.0,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,0.909,21.0,35.0,56.0,29.0,8.0,7.0,20.0,29.0,111.0,NaN,0.525,0.474,0.313,0.229,45.7,81.4,62.9,70.7,8.0,10.8,15.9,100.0,110.9,94.9,NaN,NaN,14.0,26.0,1.000,...,NaN,7.0,20.0,1.000,2.0,7.0,0.667,10.0,15.0,1.0,3.0,5.0,6.0,9.0,3.0,3.0,5.0,5.0,18.0,5.0,1.000,1.000,1.000,0.750,19.8,26.3,13.6,43.4,5.3,6.3,37.5,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True
1225,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,NaN,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,100.0,98.6,111.2,NaN,NaN,8.0,16.0,1.000,...,NaN,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.0,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,240.0,240.0,34.0,85.0,0.400,15.0,38.0,0.395,14.0,19.0,0.737,11.0,31.0,42.0,22.0,8.0,6.0,15.0,17.0,97.0,NaN,0.519,0.488,0.447,0.224,22.0,66.0,43.3,64.7,8.4,12.5,13.8,100.0,102.1,112.6,NaN,NaN,9.0,23.0,1.000,...,NaN,14.0,26.0,0.750,7.0,14.0,0.500,8.0,9.0,1.0,5.0,13.0,16.0,8.0,4.0,2.0,5.0,4.0,43.0,21.0,0.750,0.750,1.000,1.000,15.5,29.1,20.0,30.2,6.1,10.3,42.9,36.3,133.0,112.0,GSW,107,0,2022,2022-06-10,False
11533,240.0,240.0,41.0,88.0,0.466,9.0,40.0,0.225,13.0,15.0,0.867,4.0,35.0,39.0,23.0,9.0,2.0,6.0,28.0,104.0,NaN,0.550,0.517,0.455,0.170,9.3,81.4,45.3,56.1,9.4,4.7,6.0,100.0,108.4,97.9,NaN,NaN,12.0,23.0,0.750,...,NaN,10.0,20.0,1.000,5.0,9.0,1.000,8.0,10.0,1.0,3.0,10.0,10.0,4.0,1.0,2.0,5.0,4.0,27.0,11.0,1.500,1.500,1.000,4.000,84.8,84.8,42.4,100.0,2.3,12.4,45.0,94.4,300.0,112.0,BOS,94,0,2022,2022-06-13,True
11532,240.0,240.0,31.0,75.0,0.413,11.0,32.0,0.344,21.0,31.0,0.677,8.0,39.0,47.0,18.0,2.0,2.0,18.0,16.0,94.0,NaN,0.530,0.487,0.427,0.413,18.6,90.7,54.7,58.1,2.1,4.2

Sorting it only by the date makes the index all over the place now, so we will get reset the index and make sure that it does not create a duplicate column with the old index. 

In [7]:
df = df.reset_index(drop=True)  # The drop parameter allows us to drop the old index altogether and does not create a duplicate column.

In [8]:
df

,mp,mp.1,fg,fga,fg%,3p,3pa,3p%,ft,fta,ft%,orb,drb,trb,ast,stl,blk,tov,pf,pts,+/-,ts%,efg%,3par,ftr,orb%,drb%,trb%,ast%,stl%,blk%,tov%,usg%,ortg,drtg,mp_max,mp_max.1,fg_max,fga_max,fg%_max,...,mp_max_opp.1,fg_max_opp,fga_max_opp,fg%_max_opp,3p_max_opp,3pa_max_opp,3p%_max_opp,ft_max_opp,fta_max_opp,ft%_max_opp,orb_max_opp,drb_max_opp,trb_max_opp,ast_max_opp,stl_max_opp,blk_max_opp,tov_max_opp,pf_max_opp,pts_max_opp,+/-_max_opp,ts%_max_opp,efg%_max_opp,3par_max_opp,ftr_max_opp,orb%_max_opp,drb%_max_opp,trb%_max_opp,ast%_max_opp,stl%_max_opp,blk%_max_opp,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,35.0,83.0,0.422,6.0,18.0,0.333,19.0,27.0,0.704,8.0,25.0,33.0,21.0,9.0,3.0,18.0,26.0,95.0,NaN,0.501,0.458,0.217,0.325,18.6,54.3,37.1,60.0,9.0,4.5,15.9,100.0,94.9,110.9,NaN,NaN,7.0,20.0,1.000,...,NaN,14.0,26.0,1.000,5.0,12.0,1.000,7.0,7.0,1.0,5.0,5.0,9.0,7.0,2.0,2.0,7.0,5.0,40.0,20.0,1.136,1.000,0.462,0.571,39.0,27.2,25.2,50.1,19.1,7.9,69.4,43.7,206.0,104.0,GSW,111,1,2016,2015-10-27,False
1,240.0,240.0,38.0,94.0,0.404,9.0,29.0,0.310,10.0,17.0,0.588,11.0,39.0,50.0,26.0,5.0,7.0,10.0,21.0,95.0,NaN,0.468,0.452,0.309,0.181,21.6,84.8,51.5,68.4,5.0,10.3,9.0,100.0,95.5,97.5,NaN,NaN,12.0,22.0,0.571,...,NaN,8.0,22.0,0.750,3.0,5.0,0.750,5.0,5.0,1.0,2.0,8.0,10.0,5.0,2.0,6.0,4.0,6.0,19.0,9.0,0.820,0.875,0.556,1.333,12.2,38.5,26.0,30.3,2.8,14.0,53.2,34.6,162.0,104.0,CHI,97,1,2016,2015-10-27,False
2,240.0,240.0,37.0,87.0,0.425,7.0,19.0,0.368,16.0,23.0,0.696,7.0,40.0,47.0,13.0,6.0,10.0,13.0,22.0,97.0,NaN,0.499,0.466,0.218,0.264,15.2,78.4,48.5,35.1,6.0,15.4,11.8,100.0,97.5,95.5,NaN,NaN,8.0,22.0,0.750,...,NaN,12.0,22.0,0.571,3.0,7.0,0.500,3.0,4.0,1.0,4.0,10.0,12.0,7.0,1.0,2.0,3.0,4.0,25.0,9.0,0.714,0.714,1.000,2.000,10.8,41.9,23.8,31.2,2.8,18.5,30.4,29.0,138.0,105.0,CLE,95,0,2016,2015-10-27,True
3,240.0,240.0,41.0,96.0,0.427,9.0,30.0,0.300,20.0,22.0,0.909,21.0,35.0,56.0,29.0,8.0,7.0,20.0,29.0,111.0,NaN,0.525,0.474,0.313,0.229,45.7,81.4,62.9,70.7,8.0,10.8,15.9,100.0,110.9,94.9,NaN,NaN,14.0,26.0,1.000,...,NaN,7.0,20.0,1.000,2.0,7.0,0.667,10.0,15.0,1.0,3.0,5.0,6.0,9.0,3.0,3.0,5.0,5.0,18.0,5.0,1.000,1.000,1.000,0.750,19.8,26.3,13.6,43.4,5.3,6.3,37.5,38.9,201.0,120.0,NOP,95,0,2016,2015-10-27,True
4,240.0,240.0,37.0,82.0,0.451,8.0,27.0,0.296,12.0,15.0,0.800,7.0,33.0,40.0,22.0,9.0,4.0,15.0,25.0,94.0,NaN,0.530,0.500,0.329,0.183,16.3,58.9,40.4,59.5,9.4,6.0,14.5,100.0,98.6,111.2,NaN,NaN,8.0,16.0,1.000,...,NaN,7.0,19.0,0.600,4.0,7.0,0.571,6.0,10.0,1.0,8.0,11.0,19.0,5.0,2.0,2.0,3.0,4.0,21.0,26.0,0.685,0.643,0.833,0.625,18.5,41.2,24.8,35.6,3.2,4.7,33.3,23.6,132.0,104.0,DET,106,0,2016,2015-10-27,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17767,240.0,240.0,34.0,85.0,0.400,15.0,38.0,0.395,14.0,19.0,0.737,11.0,31.0,42.0,22.0,8.0,6.0,15.0,17.0,97.0,NaN,0.519,0.488,0.447,0.224,22.0,66.0,43.3,64.7,8.4,12.5,13.8,100.0,102.1,112.6,NaN,NaN,9.0,23.0,1.000,...,NaN,14.0,26.0,0.750,7.0,14.0,0.500,8.0,9.0,1.0,5.0,13.0,16.0,8.0,4.0,2.0,5.0,4.0,43.0,21.0,0.750,0.750,1.000,1.000,15.5,29.1,20.0,30.2,6.1,10.3,42.9,36.3,133.0,112.0,GSW,107,0,2022,2022-06-10,False
17768,240.0,240.0,41.0,88.0,0.466,9.0,40.0,0.225,13.0,15.0,0.867,4.0,35.0,39.0,23.0,9.0,2.0,6.0,28.0,104.0,NaN,0.550,0.517,0.455,0.170,9.3,81.4,45.3,56.1,9.4,4.7,6.0,100.0,108.4,97.9,NaN,NaN,12.0,23.0,0.750,...,NaN,10.0,20.0,1.000,5.0,9.0,1.000,8.0,10.0,1.0,3.0,10.0,10.0,4.0,1.0,2.0,5.0,4.0,27.0,11.0,1.500,1.500,1.000,4.000,84.8,84.8,42.4,100.0,2.3,12.4,45.0,94.4,300.0,112.0,BOS,94,0,2022,2022-06-13,True
17769,240.0,240.0,31.0,75.0,0.413,11.0,32.0,0.344,21.0,31.0,0.677,8.0,39.0,47.0,18.0,2.0,2.0,18.0,16.0,94.0,NaN,0.530,0.487,0.427,0.413,18.6,90.7,54.7,58.1,2.1,4.2,16.9,100.0,97.9,

There we go, we have a clean df with the indicies reset and the dates sorted. Now I am going to do a couple more steps straight from the tutorial, the code here is not going to be different, it will only start to be different later on as this is stuff to just do the data cleaning and curation, which is done really well. 

**My goal here is not to make improvements on the data cleaning and curation, rather I want to change the actual prediction parts.** I acknowledge that data is extremely important for the best results in machine learning, but that is not the purpose of this project. 

In [9]:
# In the tutorial we are told to delete the following columns:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [ ]:
# The target is the next value in the win column, the result of the NEXT game
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

# Apply the add_target function for a certain team by grouping the df by that team
df = df.groupby("team", group_keys=False).apply(add_target)
df = df.copy()

In [ ]:
# Replace all of the null occurrences of null values in the target column 
df['target'][pd.isnull(df['target'])] = 2

In [12]:
# Turn the target into a boolean integer value
df['target'] = df['target'].astype(int, errors='ignore')

In [13]:
null_counts = df.isnull().sum()  # Check to see if any columns in the datagrame have a null value. 
null_counts = null_counts[null_counts > 0]
null_counts

+/-             17772
mp_max          17772
mp_max.1        17772
+/-_opp         17772
mp_max_opp      17772
mp_max_opp.1    17772
dtype: int64

In [14]:
# Since these values are null almost always, we can just get rid of all of them
valid_columns = df.columns[~df.columns.isin(null_counts.index)]

In [15]:
df = df[valid_columns].copy()   # Copy allows us to create a new dataframe instead of getting a part of another one.

### Machine Learning

In [16]:
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeClassifier
from sklearn.model_selection import TimeSeriesSplit

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits=3)

pca = PCA(n_components=30)

In [17]:
removed_columns = ["season", "date", "won", "target", "team", "team_opp"]   # These features cant really be turned into numbers, so we will remove them for now

In [18]:
selected_colums = df.columns[~df.columns.isin(removed_columns)]

In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_colums] = scaler.fit_transform(df[selected_colums])

In [21]:
X_pca = pca.fit_transform(df[selected_colums])

In [22]:
X_pca

array([[ 0.81931369,  0.01891564, -0.48369194, ...,  0.27484003,
         0.44244693, -0.19155112],
       [ 0.68162334, -0.90072242,  0.016364  , ..., -0.1270482 ,
         0.28183199,  0.09703132],
       [-0.68162334, -0.90072242, -0.016364  , ..., -0.1270482 ,
        -0.28183199, -0.09703132],
       ...,
       [ 0.63350248,  0.28307822,  0.61667856, ...,  0.47412008,
        -0.12251657,  0.09048086],
       [-0.67669065, -0.1044606 , -0.17654961, ...,  0.16595161,
         0.10654517, -0.11580943],
       [ 0.67669065, -0.1044606 ,  0.17654961, ...,  0.16595161,
        -0.10654517,  0.11580943]], shape=(17772, 30))

In [ ]:
def backtest(data, model, predictors, start=2, step=1):
    # Split data by season. Uses past season to predict future seasons. 
    all_predictions = []

    seasons = sorted(data["season"].unique())

    # This loop goes from the third season onwards to the last season
    # The training data is split as all the seasons before, and the test is that season
    for i in range(start, len(seasons), step):
        season = seasons[i]

        train = data[data["season"] < season]
        test = data[data["season"] == season]

        model.fit(train[predictors], train['target'])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index=test.index)

        combined = pd.concat([test['target'], preds], axis=1)
        combined.columns = ["actual", "prediction"]

        all_predictions.append(combined)
    
    return pd.concat(all_predictions)